In [1]:
import os 
import re
import random
import pandas as pd
import numpy as np


import cv2 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Model,optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout,Embedding,Add
from tqdm import tqdm
from pickle import load,dump
from sklearn.feature_extraction.text import CountVectorizer

from utils import  Image_Caption,get_tier

2023-12-07 19:04:39.873870: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 19:04:39.897760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 19:04:39.897790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 19:04:39.898498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 19:04:39.903321: I tensorflow/core/platform/cpu_feature_guar

In [2]:
with open('model/word2index.pkl', 'rb') as pickle_file:
    word_index_Mapping = load(pickle_file)

word_index_Mapping['init'] = len(word_index_Mapping) + 1
index_word_Mapping = dict(map(reversed,word_index_Mapping.items()))
vocab_size = len(word_index_Mapping) + 1
print(vocab_size)

5417


In [3]:
max_caption_length = 84

In [4]:
%%time
#word embedding
glove_dir='glove/glove.6B.300d.txt'
embeddings_index = {}
embeddings_index_all = {}
f = open(glove_dir,encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index_all[word] = coefs
    if word in word_index_Mapping.keys():
        embeddings_index[word] = coefs

f.close()    


In [ ]:
%%time
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size,embedding_dim))

for word, i in word_index_Mapping.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

CPU times: user 9.5 ms, sys: 0 ns, total: 9.5 ms
Wall time: 8.88 ms


In [ ]:
def get_model():
    image_input = Input(shape=(2048,))
    x = Dropout(0.4)(image_input)
    image_encode = Dense(256,activation='relu')(x)

    text_input = Input(shape=(max_caption_length,))
    x = Embedding(vocab_size,embedding_dim,mask_zero=True)(text_input)
    x = Dropout(0.4)(x)
    text_encode = LSTM(256)(x)

    decoder_input = Add()([image_encode,text_encode])
    x = Dense(256,activation='relu')(decoder_input)
    output = Dense(vocab_size,activation='softmax')(x)
    model = Model(inputs=[image_input,text_input],outputs=output)


    model.layers[2].set_weights([embedding_matrix])
    model.layers[2].trainable = False

    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy',optimizer=optimizer)

    return model

In [ ]:
model_path = 'model/model--173--3.22.h5'
#model = get_model()
model = load_model(model_path)

2023-12-07 18:52:43.430743: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:
femodel = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet')
incp_model_feature = Model(femodel.input,femodel.layers[-2].output)

In [ ]:
im_cp = Image_Caption(images_dir='',
                      model = model,
                      fe_model=incp_model_feature,
                      w2v = embeddings_index_all,
                      tier = get_tier('tier1.txt'),
                      word_index_Mapping=word_index_Mapping,
                      index_word_Mapping=index_word_Mapping,
                      max_caption_length=max_caption_length,
                      vocab_size=vocab_size
)

In [ ]:
im_cp.caption_video('sample.mp4')

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is not the object's thread (0x2d93a5c0).
Cannot move to target thread (0x2ea1ca20)

QObject::moveToThread: Current thread (0x2ea1ca20) is n

Time passed: 0.8796718120574951


AttributeError: module 'cv2' has no attribute 'xfeatures2d_SIFT'

: 